In [1]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

Separate total sheet music into bar images

In [3]:
img = cv2.imread('probeersel/Fmttm.png') 
rows,cols = img.shape[:2]

img_struct = img.copy()

gray = cv2.cvtColor(img_struct, cv2.COLOR_BGR2GRAY)
(thresh, im_bw) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

erode_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (1,20))
dilate_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (20,1))

step1 = cv2.erode(im_bw, erode_struct, 1)
step2 = cv2.dilate(step1, dilate_struct, 1)
im_inv = cv2.bitwise_not(step2)

img_row_sum = np.sum(im_inv,axis=1).tolist()

threshold = 0.5
for i,r in enumerate(img_row_sum):
    perc = r/(255*im_inv.shape[1])
    img_row_sum[i] = perc
    
start = []
end = []
full = []
for i, val in enumerate(img_row_sum):
    if i == 0: continue
    if val==threshold:
        pass
    elif val>threshold and img_row_sum[i-1]<threshold:
        start.append(i)
        full.append(i)
    elif val<threshold and img_row_sum[i-1]>threshold:
        end.append(i)
        full.append(i)

cut = []
for i,val in enumerate(full):
    if i == 0:
        if val in start and full[i+2] in start:
            dist = full[i+2]-val
            cutloc_s = val-dist
            cut.append([max(cutloc_s,0), full[i+2]])
    elif i == len(full)-2 and val in start and full[i-1] in end and full[i+1] in end:
        dist = full[i+1] - full[i-1]
        cutloc_e = full[i+1] + dist
        cut.append([full[i-1], min(cutloc_e,len(img_row_sum)-1)])
    elif i == len(full)-1:
        break
    elif val in start and full[i-1] in end and full[i+2] in start:
        cut.append([full[i-1], full[i+2]])
        
staffs = []
for i in range(len(cut)):
    crop1 = img[cut[i][0]:cut[i][1], 0:img.shape[1]]
    staffs.append(crop1)
    
for s in staffs:
     cv2.imshow('image cropped', s); cv2.waitKey(0); cv2.destroyAllWindows();cv2.waitKey(1)

Load bar image

In [16]:
# img = cv2.imread('probeersel/Fmttm-1bar.png') 
img_bar = staffs[0]
# cv2.imshow('image bar', img_bar)

In [17]:
height,width = img_bar.shape[:2]
img_gray = cv2.cvtColor(img_bar, cv2.COLOR_BGR2GRAY)

In [18]:
img_copy = img_bar.copy()

Find barlines

In [20]:
# Canny edge detection

img_struct_hor = img_copy.copy()
hor_size = round(img_struct_hor.shape[1]/30)
hor_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (hor_size,1))

img_struct_hor3 = cv2.dilate(img_struct_hor, hor_struct, 1)
eroted = cv2.erode(img_struct_hor3, hor_struct, 3)

gray = cv2.cvtColor(eroted, cv2.COLOR_BGR2GRAY)
(thresh, img_bw) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
gray2 = cv2.cvtColor(img_bw, cv2.COLOR_GRAY2BGR)

edges2 = cv2.Canny(gray2, 100,200)
cv2.imshow('eroted', edges2); cv2.waitKey(0); cv2.destroyAllWindows();cv2.waitKey(1)

-1

In [22]:
# Find Hough lines

img_canny = eroted.copy()
lines = cv2.HoughLinesP(edges2, 1, math.pi/2, 1, None, 50, 10)

-1

In [23]:
# Label useless lines for removal and combine consecutive lines

for i, linearr in enumerate(lines):
    line = linearr[0]
    pt1 = (line[0],line[1])
    pt2 = (line[2],line[3])
    xlen1 = pt2[0]-pt1[0]
    ylen1 = pt2[1]-pt1[1]
    rot1 = math.degrees(math.atan(ylen1/xlen1))
#         print('start = %d,%d, end = %d,%d, xlen = %d, ylen = %d'%(pt1[0],pt1[1],pt2[0],pt2[1],xlen1,ylen1))
#         print(rot1)
    if abs(rot1)>=3:
        lines[i] = [[0,0,0,0]]

for i, linearr in enumerate(lines):
#         print(linearr)
    line = linearr[0]
    pt1 = (line[0],line[1])
    pt2 = (line[2],line[3])
    for j, linearr2 in enumerate(lines):
        line2 = linearr2[0]
        pt3 = (line2[0],line2[1])
        pt4 = (line2[2],line2[3])
        if (pt3[0] in range(pt1[0]+1,pt2[0]+2) and pt3[1] in range(pt2[1]-1,pt2[1]+2)):
            if pt4[0]>pt2[0]:
                xlen1 = pt2[0]-pt1[0]
                ylen1 = pt2[1]-pt1[1]
                xlen2 = pt4[0]-pt3[0]
                ylen2 = pt4[1]-pt3[1]
                rot1 = math.degrees(math.atan(ylen1/xlen1))
                rot2 = math.degrees(math.atan(ylen2/xlen2))
    #             print('%f vs. %f'%(rot1,rot2))
                if abs(rot1-rot2)<3:
                    lines[j] = [[pt1[0], pt1[1], pt4[0], pt4[1]]]
                    lines[i] = [[0,0,0,0]]
                    break
            else:
                lines[j] = [[0,0,0,0]]

for i, linearr in enumerate(lines):
    line = linearr[0]
    pt1 = (line[0],line[1])
    pt2 = (line[2],line[3])
    xlen1 = pt2[0]-pt1[0]
    if xlen1 < width/4:
        lines[i] = [[0,0,0,0]]

In [43]:
# Remove labeled lines

new_lines = []
for linearr in lines:
    line = linearr[0]
    if line.all() == 0:
        continue
    new_lines.append(line)

In [84]:
# Function for the found lines

def calc_y(line,x):
    x1 = line[0]
    y1 = line[1]
    x2 = line[2]
    y2 = line[3]
    
    y = x*(y2-y1)/(x2-x1) + ((x2*y1 - x1*y2)/(x2-x1))
    return int(y)

In [85]:
# Stretch lines across entire image

long_lines = []

for n, line in enumerate(new_lines):
    new_p1_x = 0
    new_p1_y = calc_y(line,new_p1_x)
    
    new_p2_x = width
    new_p2_y = calc_y(line,new_p2_x)
    
    long_lines.append([new_p1_x, new_p1_y, new_p2_x, new_p2_y])

In [173]:
# Remove (semi-)duplicate lines

long_lines = sorted(long_lines, key=lambda x: x[1])
unique_lines = []

for i,l in enumerate(long_lines):
    if i==0: continue
    if l[1]-long_lines[i-1][1]>3:
        unique_lines.append(long_lines[i-1])
    elif i==len(long_lines)-1:
        unique_lines.append(long_lines[i-1])

In [92]:
# Show final lines

colours = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(0,255,255),(255,0,255)]

img_canny3 = eroted.copy()
for n, line in enumerate(unique_lines):
    nr = n%len(colours)
    pt1 = (line[0],line[1])
    pt2 = (line[2],line[3])
    cv2.line(img_canny3, pt1, pt2, colours[nr], 1)
cv2.imshow('edge detection', img_canny3); cv2.waitKey(0); cv2.destroyAllWindows();cv2.waitKey(1)

-1

In [174]:
def calc_avg_distance(lines):
    s = [x[1] for x in lines]
    e = [x[3] for x in lines]
    diff_s = [j-i for i, j in zip(s[:-1], s[1:])]
    diff_e = [j-i for i, j in zip(e[:-1], e[1:])]
    ds = sum(diff_s)/len(diff_s)
    de = sum(diff_e)/len(diff_e)
    return(int((ds+de)/2))

dist = calc_avg_distance(unique_lines)

def calc_lower_lines(lines):
    # create all(4) the lines below the bar
    s = [x[1] for x in lines]
    e = [x[3] for x in lines]
    
    s_bottom = max(s)
    e_bottom = max(e)
    
    new_lines = lines.copy()
    for i in range(1,5):
        new_line = [0,s_bottom+(i*dist),width,e_bottom+(i*dist)]
        new_lines.append(new_line)
        
    return new_lines

def calc_higher_lines(lines):
    # create all(4) the lines above the bar
    s = [x[1] for x in lines]
    e = [x[3] for x in lines]
    
    s_top = min(s)
    e_top = min(e)
    
    new_lines = lines.copy()
    for i in range(1,5):
        new_line = [0,s_top-(i*dist),width,e_top-(i*dist)]
        new_lines.append(new_line)
        
    return new_lines

In [186]:
all_lines = sorted(calc_higher_lines(calc_lower_lines(unique_lines)), key=lambda x: x[1])

In [178]:
# Show final lines

colours = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(0,255,255),(255,0,255)]

img_canny3 = eroted.copy()
for n, line in enumerate(all_lines):
    nr = n%len(colours)
    pt1 = (line[0],line[1])
    pt2 = (line[2],line[3])
    cv2.line(img_canny3, pt1, pt2, colours[nr], 1)
cv2.imshow('edge detection', img_canny3); cv2.waitKey(0); cv2.destroyAllWindows();cv2.waitKey(1)

-1

Load templates

In [93]:
template_head = cv2.imread('probeersel/head-filled.png',0)
h_head,w_head = template_head.shape[::1]

In [94]:
template_body = cv2.imread('probeersel/body.png',0)
h_body,w_body = template_body.shape[::1]

In [95]:
template_flag = cv2.imread('probeersel/flag.png',0)
h_flag,w_flag = template_flag.shape[::1]

Find locations of template matches

In [96]:
res_head = cv2.matchTemplate(img_gray, template_head, cv2.TM_CCOEFF_NORMED)
threshold = 0.8
loc_head = np.where(res_head >= threshold)

head_list = []

for pt in zip(*loc_head[::-1]):
    head_list.append([pt[0], pt[1]])

In [97]:
res_body = cv2.matchTemplate(img_gray, template_body, cv2.TM_CCOEFF_NORMED)
threshold = 0.8
loc_body = np.where(res_body >= threshold)

body_list = []

for pt in zip(*loc_body[::-1]):
    body_list.append([pt[0], pt[1]])

In [98]:
res_flag = cv2.matchTemplate(img_gray, template_flag, cv2.TM_CCOEFF_NORMED)
threshold = 0.8
loc_flag = np.where(res_flag >= threshold)

flag_list = []

for pt in zip(*loc_flag[::-1]):
    flag_list.append([pt[0], pt[1]])

Remove (semi-)duplicates

In [99]:
flag_list.sort()
remove_flag = []

for i,pt in enumerate(flag_list):
    if i in remove_flag: continue
    for j in range(i+1,len(flag_list)):
        pt2 = flag_list[j]
        if pt2[0] in range(pt[0]-4,pt[0]+4) and pt2[1] in range(pt[1]-3,pt[1]+3):
            remove_flag.append(j)
            
unique_flags = []

for i in range(len(flag_list)):
    if i not in remove_flag:
        unique_flags.append(flag_list[i])

In [100]:
head_list.sort()
remove_head = []

for i,pt in enumerate(head_list):
    if i in remove_head: continue
    for j in range(i+1,len(head_list)):
        pt2 = head_list[j]
        if pt2[0] in range(pt[0]-4,pt[0]+4) and pt2[1] in range(pt[1]-3,pt[1]+3):
            remove_head.append(j)
            
unique_heads = []

for i in range(len(head_list)):
    if i not in remove_head:
        unique_heads.append(head_list[i])

Show locations of template matches in image

In [212]:
img_copy2 = img_copy.copy()
for pt in unique_heads:
    cv2.rectangle(img_copy2, (pt[0],pt[1]), (pt[0]+w_head,pt[1]+h_head),(255,0,0),1)
for pt in unique_flags:
    cv2.rectangle(img_copy2, (pt[0],pt[1]), (pt[0]+w_flag,pt[1]+h_flag),(0,0,255),1)

cv2.imshow('image bar', img_copy2)

(test for finding note pitch)

In [209]:
pitch_list = ['G6','F6','E6','D6','C6','B6','A6','G5','F5','E5','D5','C5','B5','A5','G4','F4','E4','D4']

In [214]:
def find_pitch(x,y):
    line_vals = []
    for l in all_lines:
        line_vals.append(calc_y(l,x))
    if y<min(line_vals):
        print('too high: %d'%y)
        return('Error')
    elif y>max(line_vals):
        print('too low: %d'%y)
        return('Error')
    else:
        i=0
        found = False
        while found==False:
            i = i+1
            found = (y<line_vals[i])
        
        s = line_vals[i-1]
        e = line_vals[i]
        
        diff_s = abs(y-s)
        diff_e = abs(e-y)
        diff_m = abs(diff_s-diff_e)/2
        mindiff = min(diff_s,diff_e,diff_m)
        
        if mindiff==diff_s:
            pitch = line_vals.index(s)*2
        elif mindiff==diff_e:
            pitch = line_vals.index(e)*2
        else: pitch = line_vals.index(e)*2-1
        
    return pitch_list[pitch]

In [216]:
for pt in unique_heads:
    y = int(pt[1]+(h_head/2))
    x = int(pt[0]+(w_head/2))
    
    pitch = find_pitch(x,y)
    print(pitch)

D4
F4
D4
F4
D4
D4
F4
D4
F4
D4
D4
E4
D4
E4
D4


Find connected note parts

Determine length and pitch of notes